In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import kurtosis
from scipy import fftpack as fft
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
class Patient():
    def __init__(self,cgm):
        self.cgm = cgm
    
    def preprocess(self):
        #drop rows with 30% of values missing
        self.cgm=self.cgm.loc[self.cgm.isnull().mean(axis=1)<0.3,:]
        
        #drop last column as it has many missing values for all patients
        self.cgm=self.cgm.iloc[:,:30]
        
        #reset the indices
        self.cgm.reset_index(inplace=True,drop=True)
        
        #interpolate the remaining missing values
        self.cgm.interpolate(method='polynomial',order=3,inplace=True)
        self.cgm.bfill(inplace=True)
        
    def fft(self,):
        ndarr = fft.rfft(self.cgm, n=5, axis=1)
        df= pd.DataFrame(data=ndarr)
        df.columns=['fft'+str(i) for i in range(1,df.shape[1]+1)]
        return df
        
    def rolling_mean(self,win,olap):
        df=self.cgm.rolling(window=win,axis=1).apply(np.mean).dropna(axis=1).iloc[:,::olap]
        df.columns=['rm'+str(i) for i in range(1,df.shape[1]+1)]
        return df
    
    def kurtosis(self,win,olap):
        df=self.cgm.rolling(window=win,axis=1).apply(kurtosis).dropna(axis=1).iloc[:,::olap]
        df.columns=['kt'+str(i) for i in range(1,df.shape[1]+1) ]
        return df

    def stdev(self,win,olap):
        df=self.cgm.rolling(window=win,axis=1).apply(np.std).dropna(axis=1).iloc[:,::olap]
        df.columns=['st'+str(i) for i in range(1,df.shape[1]+1)]
        return df
    
    def featureMatrix(self):
        return pd.concat([self.fft(),self.rolling_mean(10,5),self.stdev(10,5),self.kurtosis(10,5)],axis=1)
    
    def pca(self):
        self.preprocess()
        feature_matrix = self.featureMatrix()
        cols=feature_matrix.columns
        stdscaler = StandardScaler()
        mat = stdscaler.fit_transform(feature_matrix)
        p = PCA(n_components=5)
        u = pd.DataFrame(p.fit_transform(mat))
        vt = pd.DataFrame(p.components_)
        vt.columns=cols
        return u,vt

In [3]:
p1=Patient(pd.read_csv('CGMSeriesLunchPat1.csv'))
p2=Patient(pd.read_csv('CGMSeriesLunchPat2.csv'))
p3=Patient(pd.read_csv('CGMSeriesLunchPat3.csv'))
p4=Patient(pd.read_csv('CGMSeriesLunchPat4.csv'))
p5=Patient(pd.read_csv('CGMSeriesLunchPat5.csv'))

In [4]:
u,vt=p1.pca()

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this war

ValueError: Length mismatch: Expected axis has 1 elements, new values have 20 elements

In [ ]:
for i in range(vt.shape[0]):
    row_dict = dict(vt.iloc[i])
    print(sorted(row_dict,key=row_dict.get,reverse=True)[:5])